In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def hybrid_recommendations(user_id, movie_title, ratings, movies, svd, cosine_sim, n=10, content_w=0.5, collab_w=0.5):
    try:
        content_recs = get_content_recommendations(movie_title, movies, cosine_sim, n)
        collab_recs = get_collab_recommendations(user_id, ratings, movies, svd, n)
        if content_recs.empty and collab_recs.empty:
            print("Both content-based and collaborative recommendations failed.")
            return pd.DataFrame()
        merged_recs = pd.concat([content_recs, collab_recs], ignore_index=True).drop_duplicates(subset='movieId')
        if merged_recs.empty:
            print("No recommendations after merging.")
            return pd.DataFrame()
        scaler = MinMaxScaler()
        merged_recs[['content_score', 'collab_score']] = scaler.fit_transform(merged_recs[['content_score', 'collab_score']].fillna(0))
        merged_recs['hybrid_score'] = content_w * merged_recs['content_score'] + collab_w * merged_recs['collab_score']
        return merged_recs[['movieId', 'title', 'genres', 'hybrid_score']].sort_values(by='hybrid_score', ascending=False).head(n)
    except Exception as e:
        print(f"Error in hybrid recommendations: {str(e)}")
        return pd.DataFrame()

# Note: get_content_recommendations and get_collab_recommendations are assumed to be imported or defined elsewhere